# ADHD200 — Обучение (Multiclass) на подготовленных таймсериях - Iter 1

Этот ноутбук реализует три ступени:
1) **Базлайны без нейросетей**:
   - **Статический FC** (корреляции ROI×ROI) → Logistic Regression (+PCA по желанию);
   - **Динамический FC + HMM** (или KMeans), признаки: доли состояний, переключаемость, длительность.
   - **Переносимость**: Leave-One-Site-Out для статического FC.
2) **Нейросети**: LSTM/GRU/TCN+BiGRU+Self-Attn с ковариатами (возраст/пол), веса классов, метрики `acc/auc/f1`.
3) **Строгая валидация**: групповой сплит по участнику/сайту для предотвращения утечек.

В данном ноутбуке используется подготовленный набор данных из ноутбука [ADHD200_prep.ipynb](ADHD200_prep.ipynb). Для обучения нейросети используются файлы `.npy` с таймсериями. Обучение ведётся только на данных подростков из соответствующего отфильтрованного манифеста `manifest.csv`.


In [1]:
%pip install torch
%pip install hmmlearn

import re, os
from pathlib import Path

import numpy as np
import pandas as pd

from IPython.display import display

from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from hmmlearn.hmm import GaussianHMM

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /usr/local/Caskroom/miniconda/base/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /usr/local/Caskroom/miniconda/base/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
# === Параметры обучения ===

DATA_ROOT = "./Athena_flat_filtered/cohort_teen_participants"     # та же папка, что DEST_ROOT из подготовки (ADHD200_prep.ipynb, с данными подростков)
USE_AGGREGATE = False                                             # True == брать aggregate_manifest.csv, иначе manifest.csv (первый run per subject)

# Файлы с разметкой признаков (стоит указать только LABELS_CSV, если ковариаты не используются)
LABELS_CSV = "../SortedRawDataBIDS/cohort_teen_participants/participants_class_labels.tsv"      # CSV: participant_id,label (0..N_CLASSES-1)

# Ковариаты (CSV с колонками: participant_id, age, sex). sex может быть {M,F} или {0,1}
COVARS_CSV = ""                       # если пусто, ковариаты не используются
NUM_COVARS = ["age"]                  # числовые ковариаты (например, возраст)
CAT_COVARS = ["sex"]                  # категориальные ковариаты (например, пол {M,F})
USE_COVARS = False                    # использовать ли ковариаты в модели

GROUP_SPLIT_BY = 'participant'        # 'participant' | 'site'
VAL_SIZE = 0.2                        # доля валидационного сета при GroupShuffleSplit - 20%
SEED = 42
CHECKPOINT_DIR = str(Path(DATA_ROOT)/'checkpoints')

REUSE_MODELS = False                  # если True, не будем переобучать модели, если чекпоинты уже существуют
N_CLASSES = 4                         # число классов для классификации: Inattentive, Hyperactive-Impulsive, Combined, Neurotypical
EPOCHS = 25                           # число эпох для обучения модели
BATCH_TRAIN = 32                      # размер батча для обучения (на данный момент не используется)
BATCH_VAL = 64                        # размер батча для валидации (на данный момент не используется)
LR = 1e-3
WEIGHT_DECAY = 1e-4

# === Параметры предобработки признаков ===
RUN_BASELINES = True              # запускать базлайны (FC/HMM)
FC_PCA_COMPONENTS = 200           # 0 = без PCA; иначе число компонент для PCA по FC-признакам
HMM_N_STATES = 5                  # число скрытых состояний HMM
WIN_DYN = 30                      # окно (в TR) для динамического FC
STEP_DYN = 5                      # шаг (в TR) для окна динамического FC
DO_LOSO = True                    # Leave-One-Site-Out оценка для статического FC

# Определяем девайс для обучения (CUDA / CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Обучение моделей будет произведено на {device}")


Обучение моделей будет произведено на cpu


## Загрузка манифеста и меток; подготовка групп
В этом разделе мы:
1) Загружаем `aggregate_manifest.csv` (или `manifest.csv`) и таблицу меток `LABELS_CSV`.
2) Объединяем их по `participant_id`.
3) Извлекаем **группы** для разбиения без утечки: либо по `participant`, либо по `site` (префикс в `sub-...`).
4) Делаем **GroupShuffleSplit** на train/val, чтобы разные прогоны одного участника (или весь сайт) не пересекались между сплитами.


In [3]:
root = Path(DATA_ROOT)
man_path = root/('aggregate_manifest.csv' if USE_AGGREGATE else 'manifest.csv')
manifest = pd.read_csv(man_path)
if not USE_AGGREGATE:
    manifest = manifest.sort_values('npy_path').groupby('participant_id').first().reset_index()[['participant_id','npy_path']]
display(manifest.head())

labels_df = pd.read_csv(LABELS_CSV, sep='\t') # Читаем .tsv
data_df = manifest.merge(labels_df, on='participant_id', how='inner')
print('Сэмплов (есть метки):', len(data_df))

def pid2site(pid):
    s = pid.replace('sub-','')
    m = re.match(r'([a-z]+)', s)
    return m.group(1) if m else 'na'

if GROUP_SPLIT_BY == 'site':
    data_df['group'] = data_df['participant_id'].map(pid2site)
else:
    data_df['group'] = data_df['participant_id']

idx = np.arange(len(data_df))
gss = GroupShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=SEED)
train_idx, val_idx = next(gss.split(idx, groups=data_df['group'].values))
train_df, val_df = data_df.iloc[train_idx].reset_index(drop=True), data_df.iloc[val_idx].reset_index(drop=True)

print(f"train N={len(train_df)}, val N={len(val_df)}; groups → train:{train_df['group'].nunique()}, val:{val_df['group'].nunique()}")


,participant_id,npy_path
0,sub-kki1018959,Athena_flat/sub-kki1018959/func/sfnwmrda101895...
1,sub-kki1019436,Athena_flat/sub-kki1019436/func/sfnwmrda101943...
2,sub-kki1594156,Athena_flat/sub-kki1594156/func/sfnwmrda159415...
3,sub-kki1623716,Athena_flat/sub-kki1623716/func/sfnwmrda162371...
4,sub-kki2026113,Athena_flat/sub-kki2026113/func/sfnwmrda202611...


Сэмплов (есть метки): 165
train N=132, val N=33; groups → train:132, val:33


## Ковариаты: чтение, препроцессинг, трансформация
Если указан `COVARS_CSV`, мы добавляем дополнительные предикторы (например, **age**, **sex**):
- Числовые признаки нормируем (StandardScaler).
- Категориальные — one-hot (OneHotEncoder).
Пайплайн prefit-ится только на train выборку, затем применяется к валидационной (val). Размерность кодировки передаётся в модель (`cov_dim`).


In [4]:
if USE_COVARS and COVARS_CSV:
    cov_df = pd.read_csv(COVARS_CSV)
    if 'sex' in cov_df.columns:
        cov_df['sex'] = cov_df['sex'].astype(str).str.strip().str.lower().map({'m':'M','male':'M','0':'M','1':'F','f':'F','female':'F'}).fillna(cov_df['sex'])
    pre = ColumnTransformer([
        ("num", StandardScaler(), NUM_COVARS),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse=True), CAT_COVARS)
    ], remainder='drop')
    cov_train = train_df[['participant_id']].merge(cov_df, on='participant_id', how='left')
    cov_val   = val_df[['participant_id']].merge(cov_df, on='participant_id', how='left')
    Z_train = pre.fit_transform(cov_train)
    Z_val   = pre.transform(cov_val)
    if hasattr(Z_train, 'toarray'): Z_train = Z_train.toarray()
    if hasattr(Z_val, 'toarray'):   Z_val   = Z_val.toarray()
    cov_dim = Z_train.shape[1]
else:
    Z_train = None; Z_val = None; cov_dim = 0
print('cov_dim =', cov_dim)

cov_dim = 0


## Модели: LSTM / GRU / TCN+BiGRU+Self-Attn (SE, FiLM)
- **LSTM/GRU**: компактные RNN-блоки. Ковариаты (если есть) конкатенируются к финальному скрытому состоянию.
- **ROISequenceNet (TCN+BiGRU+Attn+SE+FiLM)**: темпоральные свёртки (локальные паттерны) → двунаправленный GRU (дальние зависимости) → self-attention и SE-внимание по ROI; ковариаты подаются через FiLM.


In [5]:
# Простая LSTM модель (для сравнения с целевой)
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden=64, num_layers=1, num_classes=2, cov_dim=0):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, num_layers=num_layers, batch_first=True)
        if cov_dim>0:
            self.cov = nn.Sequential(nn.Linear(cov_dim, 64), nn.ReLU())
            self.head = nn.Sequential(nn.Linear(hidden+64, 64), nn.ReLU(), nn.Linear(64, num_classes))
        else:
            self.cov = None
            self.head = nn.Sequential(nn.Linear(hidden, 64), nn.ReLU(), nn.Linear(64, num_classes))
    def forward(self, x, z=None):
        out, (h, c) = self.lstm(x)
        h_last = h[-1]
        if self.cov is not None and z is not None:
            cz = self.cov(z)
            h_last = torch.cat([h_last, cz], dim=1)
        return self.head(h_last)

# Простая GRU модель (для сравнения с целевой)
class SimpleGRU(nn.Module):
    def __init__(self, input_size, hidden=64, num_layers=1, num_classes=2, cov_dim=0):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden, num_layers=num_layers, batch_first=True)
        if cov_dim>0:
            self.cov = nn.Sequential(nn.Linear(cov_dim, 64), nn.ReLU())
            self.head = nn.Sequential(nn.Linear(hidden+64, 64), nn.ReLU(), nn.Linear(64, num_classes))
        else:
            self.cov = None
            self.head = nn.Sequential(nn.Linear(hidden, 64), nn.ReLU(), nn.Linear(64, num_classes))
    def forward(self, x, z=None):
        out, h = self.gru(x)
        h_last = h[-1]
        if self.cov is not None and z is not None:
            cz = self.cov(z)
            h_last = torch.cat([h_last, cz], dim=1)
        return self.head(h_last)

# === Сложная многосоставная (целевая) модель TCN+BiGRU+Self-Attn (SE, FiLM) ===

class Chomp1d(nn.Module):
    def __init__(self, chomp_size): super().__init__(); self.chomp_size=chomp_size
    def forward(self, x): return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, dropout=0.1):
        super().__init__()
        pad = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size, padding=pad, dilation=dilation)
        self.chomp1 = Chomp1d(pad)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size, padding=pad, dilation=dilation)
        self.chomp2 = Chomp1d(pad)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout)
        self.down = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
    def forward(self, x):
        out = self.chomp1(self.conv1(x)); out = self.relu(out); out = self.dropout(out)
        out = self.chomp2(self.conv2(out)); out = self.relu(out); out = self.dropout(out)
        return self.relu(out + self.down(x))

class SE1D(nn.Module):
    def __init__(self, channels, r=8):
        super().__init__()
        self.fc1 = nn.Linear(channels, max(1, channels // r))
        self.fc2 = nn.Linear(max(1, channels // r), channels)
    def forward(self, x):
        s = x.mean(dim=2); w = torch.sigmoid(self.fc2(F.relu(self.fc1(s))))
        return x * w.unsqueeze(-1)

class FiLM(nn.Module):
    def __init__(self, cov_dim, feat_ch):
        super().__init__(); self.gamma = nn.Linear(cov_dim, feat_ch); self.beta = nn.Linear(cov_dim, feat_ch)
    def forward(self, x, z):
        g = self.gamma(z).unsqueeze(-1); b = self.beta(z).unsqueeze(-1); return x * (1 + g) + b

class MultiHeadTemporalAttention(nn.Module):
    def __init__(self, d_model, n_heads=4):
        super().__init__(); self.mha = nn.MultiheadAttention(d_model, n_heads, batch_first=True)
        self.ln = nn.LayerNorm(d_model); self.ff = nn.Sequential(nn.Linear(d_model,2*d_model), nn.ReLU(), nn.Linear(2*d_model,d_model))
    def forward(self, X, mask=None):
        key_padding_mask = (~mask.bool()) if mask is not None else None
        Y,_ = self.mha(X,X,X, key_padding_mask=key_padding_mask)
        X = self.ln(X+Y); Z = self.ln(X + self.ff(X))
        if mask is None: return Z.mean(dim=1)
        w = mask.float().unsqueeze(-1); return (Z*w).sum(dim=1)/(w.sum(dim=1)+1e-8)

class ROISequenceNet(nn.Module):
    def __init__(self, n_roi, n_classes=4, cov_dim=0, tcn_dropout=0.1, gru_hidden=128, attn_heads=4):
        super().__init__()
        self.film = FiLM(cov_dim, 128) if cov_dim>0 else None
        self.se1 = SE1D(n_roi)
        self.tcn = nn.Sequential(
            TemporalBlock(n_roi, 128, kernel_size=5, dilation=1, dropout=tcn_dropout),
            TemporalBlock(128, 128, kernel_size=5, dilation=2, dropout=tcn_dropout),
            TemporalBlock(128, 128, kernel_size=5, dilation=4, dropout=tcn_dropout),
        )
        self.se2 = SE1D(128)
        self.bigru = nn.GRU(input_size=128, hidden_size=128, batch_first=True, bidirectional=True)
        self.attn = MultiHeadTemporalAttention(d_model=256, n_heads=4)
        self.head = nn.Sequential(nn.Linear(256,128), nn.ReLU(), nn.Dropout(0.2), nn.Linear(128, n_classes))
    def forward(self, x, mask=None, z=None):
        x = x.transpose(1,2); x = self.se1(x)
        if self.film is not None and z is not None: x = self.film(x, z)
        x = self.tcn(x); x = self.se2(x); x = x.transpose(1,2)
        y,_ = self.bigru(x); emb = self.attn(y, mask=mask); return self.head(emb)


## Датасеты, обучение и метрики


### Датасет NPYDataset: загрузка таймсерий из `.npy` файлов по пути, указанному в манифесте.
- При инициализации загружаются пути к `.npy` файлам и метки.
- При вызове `__getitem__` загружается соответствующий файл и возвращаются данные и метка.

In [6]:
# Создадим датасет для обучения моделей
class NPYDataset(Dataset):
    def __init__(self, df, Z=None):
        self.df = df.reset_index(drop=True)
        self.Z  = Z

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        row = self.df.iloc[i]
        X = np.load(row['npy_path']).astype('float32')
        y = int(row['label'])
        if self.Z is not None:
            z = self.Z[i].astype('float32')
            return torch.from_numpy(X), torch.from_numpy(z), torch.tensor(y)
        else:
            return torch.from_numpy(X), torch.empty(0), torch.tensor(y)
        
cls_counts = train_df['label'].value_counts().sort_index()
print("Классы в обучающем сете:")
for c,count in cls_counts.items():
    print(f"  Класс {c}: {count} сэмплов")

# Создадим загрузчики данных
# train_dl и val_dl будут использоваться для обучения и валидации моделей соответственно
train_dl = DataLoader(NPYDataset(train_df, Z_train), batch_size=32, shuffle=True)
val_dl   = DataLoader(NPYDataset(val_df,   Z_val),   batch_size=64, shuffle=False)

sample_X = np.load(train_df.iloc[0]['npy_path'])
T,R = sample_X.shape

print(f"Пример формы данных: T={T}, R={R} (временные точки x ROI)")

Классы в обучающем сете:
  Класс 0: 95 сэмплов
  Класс 1: 17 сэмплов
  Класс 2: 2 сэмплов
  Класс 3: 18 сэмплов
Пример формы данных: T=120, R=351 (временные точки x ROI)


### Организация обучения
- Используется стандартный цикл обучения PyTorch с оптимизатором Adam и планировщиком lr.ReduceLROnPlateau.
- Метрики: accuracy, precision, AUC, F1-score (макро).
- Веса классов учитываются в функции потерь (CrossEntropyLoss).
- Последовательно обучаем модели LSTM, GRU и ROISequenceNet, сравнивая их производительность на валидационном наборе.

In [7]:
# Универсальный класс тренировки модели
from sklearn.metrics import precision_score

class Trainer:
    def __init__(self, model, train_dl, val_dl, class_counts: pd.Series, device, lr=1e-3, weight_decay=1e-4):
        self.model = model.to(device)
        self.train_dl = train_dl
        self.val_dl = val_dl
        self.device = device
        # Явный список меток (для случаев когда на валидации отсутствует класс)
        self.class_labels = sorted(class_counts.index.tolist())
        self.class_count = len(self.class_labels)
        # Взвешенная кросс-энтропия для несбалансированных классов
        weights = (1.0 / (class_counts + 1e-9))
        weights = (weights / weights.sum()) * len(class_counts)
        class_weights = torch.tensor(weights.values, dtype=torch.float32).to(device)
        self.criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        for X, Z, y in self.train_dl:
            X, y = X.to(self.device), y.to(self.device)
            if Z.nelement() > 0:
                Z = Z.to(self.device)
                outputs = self.model(X, z=Z)
            else:
                outputs = self.model(X)
            loss = self.criterion(outputs, y)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item() * X.size(0)
        return total_loss / len(self.train_dl.dataset)

    def evaluate(self):
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_labels = []
        all_probs = []
        with torch.no_grad():
            for X, Z, y in self.val_dl:
                X, y = X.to(self.device), y.to(self.device)
                if Z.nelement() > 0:
                    Z = Z.to(self.device)
                    outputs = self.model(X, z=Z)
                else:
                    outputs = self.model(X)
                loss = self.criterion(outputs, y)
                total_loss += loss.item() * X.size(0)
                probs = F.softmax(outputs, dim=1)
                preds = torch.argmax(probs, dim=1)
                all_probs.append(probs.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y.cpu().numpy())
        all_probs = np.vstack(all_probs)
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        # AUC: если на валидации отсутствует какой-то класс, передаём полный список labels
        try:
            auc = roc_auc_score(all_labels, all_probs, average='weighted', multi_class='ovr', labels=self.class_labels)
        except ValueError:
            # Если остаётся только один класс или другая проблема — ставим NaN
            auc = float('nan')
        f1 = f1_score(all_labels, all_preds, average='weighted')
        return total_loss / len(self.val_dl.dataset), accuracy, precision, auc, f1

In [29]:
# Функция для обучения модели
def train_model(model, train_dl, val_dl, class_counts, device, epochs=10, checkpoint_dir=None, lr=1e-3, weight_decay=1e-4):
    # Создадим директорию для чекпоинтов если нужно
    if checkpoint_dir and not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir, exist_ok=True)
    # Инициализируем тренер
    trainer = Trainer(model, train_dl, val_dl, class_counts, device, lr, weight_decay)
    # Запускаем обучение
    best_acc = 0.0
    for epoch in range(epochs):
        train_loss = trainer.train_epoch()
        val_loss, val_acc, val_prec, val_auc, val_f1 = trainer.evaluate()
        print(f"Эпоха {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Val Prec: {val_prec:.4f} | Val AUC: {val_auc:.4f} | Val F1: {val_f1:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), os.path.join(checkpoint_dir, 'best_model.pth'))
            print('  ↳ saved best checkpoint (Accuracy).')
    print("Лучший валид. Accuracy:", best_acc)

### Обучение Simple LSTM

In [34]:
model_lstm = SimpleLSTM(R, hidden=64, num_layers=1, num_classes=N_CLASSES, cov_dim=cov_dim).to(device)

# Создадим директорию для чекпоинтов
checkpoint_dir_path_lstm = f"{CHECKPOINT_DIR}/SimpleLSTM"
os.makedirs(checkpoint_dir_path_lstm, exist_ok=True)

# Запустим обучение модели, если не нужно переиспользовать существующую
if not REUSE_MODELS or not os.path.exists(os.path.join(checkpoint_dir_path_lstm, 'best_model.pth')):
    train_model(model_lstm, train_dl, val_dl, cls_counts, device, epochs=EPOCHS, checkpoint_dir=checkpoint_dir_path_lstm, lr=LR, weight_decay=WEIGHT_DECAY)
else:
    model_lstm.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_lstm, 'best_model.pth')))
    print("Загружен существующий чекпоинт SimpleLSTM.")

# Статистика обучения SimpleLSTM
val_loss_lstm, val_acc_lstm, val_prec_lstm, val_auc_lstm, val_f1_lstm = Trainer(model_lstm, train_dl, val_dl, cls_counts, device).evaluate()
print(f"SimpleLSTM - Val Acc: {val_acc_lstm:.4f} | Val F1: {val_f1_lstm:.4f} | Val AUC: {val_auc_lstm:.4f}")

# Лучшая модель SimpleLSTM:
print("Лучшая модель SimpleLSTM:")
model_lstm.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_lstm, 'best_model.pth')))
model_lstm.eval()
val_loss_lstm, val_acc_lstm, val_prec_lstm, val_auc_lstm, val_f1_lstm = Trainer(model_lstm, train_dl, val_dl, cls_counts, device).evaluate()
print(f"Лучшая модель SimpleLSTM - Val Acc: {val_acc_lstm:.4f} | Val F1: {val_f1_lstm:.4f} | Val AUC: {val_auc_lstm:.4f}")

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 1/25 - Train Loss: 1.3872 | Val Loss: 1.4011 | Val Acc: 0.6061 | Val Prec: 0.4242 | Val AUC: 0.5662 | Val F1: 0.4991
  ↳ saved best checkpoint (Accuracy).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 2/25 - Train Loss: 1.2994 | Val Loss: 1.3974 | Val Acc: 0.6364 | Val Prec: 0.4311 | Val AUC: 0.5658 | Val F1: 0.5140
  ↳ saved best checkpoint (Accuracy).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


...

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 25/25 - Train Loss: 0.0223 | Val Loss: 2.5266 | Val Acc: 0.6061 | Val Prec: 0.5684 | Val AUC: 0.5191 | Val F1: 0.5341
Лучший валид. Accuracy: 0.6666666666666666


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


SimpleLSTM - Val Acc: 0.6061 | Val F1: 0.5341 | Val AUC: 0.5191
Лучшая модель SimpleLSTM:
Лучшая модель SimpleLSTM - Val Acc: 0.6667 | Val F1: 0.5792 | Val AUC: 0.5766


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


### Обучение модели Simple GRU

In [35]:
model_gru = SimpleGRU(R, hidden=64, num_layers=1, num_classes=N_CLASSES, cov_dim=cov_dim).to(device)

# Создадим директорию для чекпоинтов
checkpoint_dir_path_gru = f"{CHECKPOINT_DIR}/SimpleGRU"
os.makedirs(checkpoint_dir_path_gru, exist_ok=True)

# Запустим обучение модели, если не нужно переиспользовать существующую
if not REUSE_MODELS or not os.path.exists(os.path.join(checkpoint_dir_path_gru, 'best_model.pth')):
    train_model(model_gru, train_dl, val_dl, cls_counts, device, epochs=EPOCHS, checkpoint_dir=checkpoint_dir_path_gru, lr=LR, weight_decay=WEIGHT_DECAY)
else:
    model_gru.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_gru, 'best_model.pth')))
    print("Загружен существующий чекпоинт SimpleGRU.")

# Статистика обучения SimpleGRU
val_loss_gru, val_acc_gru, val_prec_gru, val_auc_gru, val_f1_gru = Trainer(model_gru, train_dl, val_dl, cls_counts, device).evaluate()
print(f"SimpleGRU - Val Acc: {val_acc_gru:.4f} | Val F1: {val_f1_gru:.4f} | Val AUC: {val_auc_gru:.4f}")

# Лучшая модель SimpleGRU:
print("Лучшая модель SimpleGRU:")
model_gru.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_gru, 'best_model.pth')))
model_gru.eval()
val_loss_gru, val_acc_gru, val_prec_gru, val_auc_gru, val_f1_gru = Trainer(model_gru, train_dl, val_dl, cls_counts, device).evaluate()
print(f"Лучшая модель SimpleGRU - Val Acc: {val_acc_gru:.4f} | Val F1: {val_f1_gru:.4f} | Val AUC: {val_auc_gru:.4f}")

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 1/25 - Train Loss: 1.3944 | Val Loss: 1.3276 | Val Acc: 0.3636 | Val Prec: 0.4909 | Val AUC: 0.5836 | Val F1: 0.4047
  ↳ saved best checkpoint (Accuracy).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 2/25 - Train Loss: 1.2547 | Val Loss: 1.3194 | Val Acc: 0.3939 | Val Prec: 0.4935 | Val AUC: 0.5963 | Val F1: 0.4276
  ↳ saved best checkpoint (Accuracy).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


...

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 23/25 - Train Loss: 0.0448 | Val Loss: 1.8231 | Val Acc: 0.6061 | Val Prec: 0.5303 | Val AUC: 0.5831 | Val F1: 0.5483


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 24/25 - Train Loss: 0.0379 | Val Loss: 1.8975 | Val Acc: 0.6061 | Val Prec: 0.5303 | Val AUC: 0.5692 | Val F1: 0.5483
Эпоха 25/25 - Train Loss: 0.0312 | Val Loss: 1.9646 | Val Acc: 0.6061 | Val Prec: 0.5303 | Val AUC: 0.5717 | Val F1: 0.5483
Лучший валид. Accuracy: 0.6363636363636364
SimpleGRU - Val Acc: 0.6061 | Val F1: 0.5483 | Val AUC: 0.5717
Лучшая модель SimpleGRU:
Лучшая модель SimpleGRU - Val Acc: 0.6364 | Val F1: 0.5985 | Val AUC: 0.6226


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: Und

### Обучение модели ROISequenceNet

In [36]:
model_roi_seq_net = ROISequenceNet(n_roi=R, n_classes=N_CLASSES, cov_dim=cov_dim).to(device)

# Создадим директорию для чекпоинтов
checkpoint_dir_path_roi_seq_net = f"{CHECKPOINT_DIR}/ROISequenceNet"
os.makedirs(checkpoint_dir_path_roi_seq_net, exist_ok=True)

# Запустим обучение модели, если не нужно переиспользовать существующую
if not REUSE_MODELS or not os.path.exists(os.path.join(checkpoint_dir_path_roi_seq_net, 'best_model.pth')):
    train_model(model_roi_seq_net, train_dl, val_dl, cls_counts, device, epochs=EPOCHS, checkpoint_dir=checkpoint_dir_path_roi_seq_net, lr=LR, weight_decay=WEIGHT_DECAY)
else:
    model_roi_seq_net.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_roi_seq_net, 'best_model.pth')))
    print("Загружен существующий чекпоинт ROISequenceNet.")

# Статистика обучения ROISequenceNet
val_loss_roi_seq_net, val_acc_roi_seq_net, val_prec_roi_seq_net, val_auc_roi_seq_net, val_f1_roi_seq_net = Trainer(model_roi_seq_net, train_dl, val_dl, cls_counts, device).evaluate()
print(f"ROISequenceNet - Val Acc: {val_acc_roi_seq_net:.4f} | Val F1: {val_f1_roi_seq_net:.4f} | Val AUC: {val_auc_roi_seq_net:.4f}")

# Лучшая модель ROISequenceNet:
print("Лучшая модель ROISequenceNet:")
model_roi_seq_net.load_state_dict(torch.load(os.path.join(checkpoint_dir_path_roi_seq_net, 'best_model.pth')))
model_roi_seq_net.eval()
val_loss_roi_seq_net, val_acc_roi_seq_net, val_prec_roi_seq_net, val_auc_roi_seq_net, val_f1_roi_seq_net = Trainer(model_roi_seq_net, train_dl, val_dl, cls_counts, device).evaluate()
print(f"Лучшая модель ROISequenceNet - Val Acc: {val_acc_roi_seq_net:.4f} | Val F1: {val_f1_roi_seq_net:.4f} | Val AUC: {val_auc_roi_seq_net:.4f}")

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 1/25 - Train Loss: 1.5645 | Val Loss: 1.4332 | Val Acc: 0.6364 | Val Prec: 0.4050 | Val AUC: 0.6993 | Val F1: 0.4949
  ↳ saved best checkpoint (Accuracy).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 2/25 - Train Loss: 1.5947 | Val Loss: 1.2199 | Val Acc: 0.6364 | Val Prec: 0.4050 | Val AUC: 0.4452 | Val F1: 0.4949


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


...

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Эпоха 25/25 - Train Loss: 0.8985 | Val Loss: 1.1064 | Val Acc: 0.3030 | Val Prec: 0.7304 | Val AUC: 0.5151 | Val F1: 0.1977
Лучший валид. Accuracy: 0.6363636363636364


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


ROISequenceNet - Val Acc: 0.3030 | Val F1: 0.1977 | Val AUC: 0.5151
Лучшая модель ROISequenceNet:
Лучшая модель ROISequenceNet - Val Acc: 0.6364 | Val F1: 0.4949 | Val AUC: 0.6993


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


## Baseline A — Статический функциональный коннектом (FC) + логистическая регрессия
Идея: из каждого временного ряда NPY `(T,R)` считаем корреляционную матрицу `R×R` (Пирсон), применяем **Фишера z** и векторизуем верхний треугольник → получаем признак фиксированной длины. 
Затем обучаем **LogisticRegression (multinomial)**, опционально предваряя **PCA** до `FC_PCA_COMPONENTS`.
Разбиение — то же групповое (без утечек).

In [12]:
if RUN_BASELINES:
    def fisher_z(r):
        r = np.clip(r, -0.999999, 0.999999)
        return 0.5*np.log((1+r)/(1-r))

    def vec_uppertri(M):
        iu = np.triu_indices_from(M, k=1)
        return M[iu]

    def fc_vector_from_npy(path):
        X = np.load(path)  # (T,R)
        C = np.corrcoef(X, rowvar=False)
        # Replace NaNs arising from zero-variance ROI time series
        C = np.nan_to_num(C, nan=0.0, posinf=0.0, neginf=0.0)
        np.fill_diagonal(C, 1.0)
        Cz = fisher_z(C)
        return vec_uppertri(Cz)

    def build_fc_matrix(df):
        feats = [fc_vector_from_npy(p) for p in df['npy_path']]
        X = np.vstack(feats)
        # Safety: ensure no NaNs remain
        X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
        y = df['label'].values.astype(int)
        return X, y

    Xtr, ytr = build_fc_matrix(train_df)
    Xva, yva = build_fc_matrix(val_df)
    scaler = StandardScaler(with_mean=True, with_std=True)
    Xtr_s = scaler.fit_transform(Xtr)
    Xva_s = scaler.transform(Xva)
    # Extra safety before PCA
    if np.isnan(Xtr_s).any() or np.isnan(Xva_s).any():
        Xtr_s = np.nan_to_num(Xtr_s, nan=0.0, posinf=0.0, neginf=0.0)
        Xva_s = np.nan_to_num(Xva_s, nan=0.0, posinf=0.0, neginf=0.0)
    
    if FC_PCA_COMPONENTS and FC_PCA_COMPONENTS > 0:
        # PCA ограничение: n_components <= min(n_samples, n_features)
        allowable = min(Xtr_s.shape[0], Xtr_s.shape[1])
        n_comp = min(FC_PCA_COMPONENTS, allowable)
        if n_comp < 1:
            print("Пропуск PCA: недостаточно допустимых компонент.")
        else:
            if n_comp != FC_PCA_COMPONENTS:
                print(f"Понижено число компонент PCA с {FC_PCA_COMPONENTS} до {n_comp} (min(n_samples={Xtr_s.shape[0]}, n_features={Xtr_s.shape[1]})).")
            pca = PCA(n_components=n_comp)
            Xtr_s = pca.fit_transform(Xtr_s)
            Xva_s = pca.transform(Xva_s)
    clf = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs')
    clf.fit(Xtr_s, ytr)
    pr_tr = clf.predict_proba(Xtr_s); pr_va = clf.predict_proba(Xva_s)
    yh_va = pr_va.argmax(1)
    acc = accuracy_score(yva, yh_va)
    f1  = f1_score(yva, yh_va, average='macro')
    try:
        auc = roc_auc_score(yva, pr_va, multi_class='ovr', average='macro')
    except Exception:
        auc = float('nan')
    print(f"[FC-LogReg] val_acc={acc:.3f} | val_auc={auc:.3f} | val_f1={f1:.3f}")


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Понижено число компонент PCA с 200 до 132 (min(n_samples=132, n_features=61425)).
[FC-LogReg] val_acc=0.667 | val_auc=nan | val_f1=0.325


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## Базлайн B — Динамический FC + HMM-фичи (или KMeans fallback)
Идея: скользящим окном (`WIN_DYN`, шаг `STEP_DYN`) считаем FC по каждому окну → получаем последовательность векторов ("кадров") на субъекта. 
На **train** обучаем `GaussianHMM(n_states=HMM_N_STATES)` по этим кадрам. Для каждого субъекта считаем признаки:
- доля времени в каждом состоянии,
- частота переключений,
- средняя длительность посещений.
Если `hmmlearn` недоступен, используем **KMeans** как приближение скрытых состояний.
Классификатор: `LogisticRegression` на агрегированных признаках.

In [13]:
def sliding_windows(X, win=30, step=5):
    T, R = X.shape
    for s in range(0, max(1, T - win + 1), step):
        e = min(T, s + win)
        if e - s >= max(5, win//2):
            yield X[s:e]

def fc_frames_from_npy(path, win=30, step=5):
    X = np.load(path)
    frames = []
    for seg in sliding_windows(X, win, step):
        C = np.corrcoef(seg, rowvar=False)
        r = 0.5*np.log((1+np.clip(C, -0.999999, 0.999999))/(1-np.clip(C, -0.999999, 0.999999)))
        iu = np.triu_indices_from(r, k=1)
        frames.append(r[iu])
    if not frames:
        C = np.corrcoef(X, rowvar=False); iu = np.triu_indices_from(C, 1)
        r = 0.5*np.log((1+np.clip(C, -0.999999, 0.999999))/(1-np.clip(C, -0.999999, 0.999999)))
        frames = [r[iu]]
    return np.vstack(frames)

def summarize_states(states, n_states):
    T = len(states)
    if T == 0:
        return np.zeros(n_states + 2)
    frac = np.bincount(states, minlength=n_states) / max(1, T)
    switches = (states[1:] != states[:-1]).mean() if T>1 else 0.0
    lengths = []
    cur = 1
    for i in range(1, T):
        if states[i]==states[i-1]: cur += 1
        else: lengths.append(cur); cur = 1
    lengths.append(cur)
    dwell = float(np.mean(lengths))
    return np.concatenate([frac, [switches, dwell]])

In [14]:
if RUN_BASELINES:
    # HAS_HMM флаг (модуль уже импортирован выше, просто оставим структуру)
    try:
        HAS_HMM = True # По-умолчанию используем Hidden Markov Model
    except Exception:
        HAS_HMM = False

    # Получаем кадры динамического FC
    tr_frames = [fc_frames_from_npy(p, WIN_DYN, STEP_DYN) for p in train_df['npy_path']]
    va_frames = [fc_frames_from_npy(p, WIN_DYN, STEP_DYN) for p in val_df['npy_path']]

    # Санитизация (NaN / inf -> 0) для устойчивости HMM / KMeans
    def sanitize_frames(fr_list):
        return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in fr_list]

    tr_frames = sanitize_frames(tr_frames)
    va_frames = sanitize_frames(va_frames)

    # Объединяем обучающие кадры
    Xtrain_seq = np.vstack(tr_frames)
    Xtrain_seq = np.nan_to_num(Xtrain_seq, nan=0.0, posinf=0.0, neginf=0.0)

    n_states = int(HMM_N_STATES)

    if HAS_HMM:
        model_states = GaussianHMM(n_components=n_states, covariance_type='diag', n_iter=200, random_state=0)
        model_states.fit(Xtrain_seq)

        def assign_states(frames):
            frames = np.nan_to_num(frames, nan=0.0, posinf=0.0, neginf=0.0)
            return model_states.predict(frames)
    else:
        kmeans = KMeans(n_clusters=n_states, random_state=0).fit(Xtrain_seq)

        def assign_states(frames):
            frames = np.nan_to_num(frames, nan=0.0, posinf=0.0, neginf=0.0)
            return kmeans.predict(frames)
        
    def build_dyn_feats(frames_list):
        feats = []
        for fr in frames_list:
            st = assign_states(fr)
            feats.append(summarize_states(st, n_states))
        return np.vstack(feats)

    # Строим агрегированные признаки состояний
    Xtr = build_dyn_feats(tr_frames); ytr = train_df['label'].values.astype(int)
    Xva = build_dyn_feats(va_frames); yva = val_df['label'].values.astype(int)

    # Финальная модель
    clf = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs')
    clf.fit(Xtr, ytr)

    pr_va = clf.predict_proba(Xva); yh_va = pr_va.argmax(1)
    acc = accuracy_score(yva, yh_va)
    f1  = f1_score(yva, yh_va, average='macro')
    try:
        auc = roc_auc_score(yva, pr_va, multi_class='ovr', average='macro')
    except Exception:
        auc = float('nan')
    print(f"[DynFC-HMM] val_acc={acc:.3f} | val_auc={auc:.3f} | val_f1={f1:.3f} | HAS_HMM={HAS_HMM}")


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[DynFC-HMM] val_acc=0.636 | val_auc=nan | val_f1=0.259 | HAS_HMM=True


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## Оценка переносимости: Leave-One-Site-Out (LOSO) для FC-baseline
Для каждого сайта извлекаем его как **валидационную** выборку, обучаемся на остальных и считаем метрики. Это имитирует перенос на новый центр/сканер.

In [15]:
def fisher_z(r):
    r = np.clip(r, -0.999999, 0.999999)
    return 0.5*np.log((1+r)/(1-r))

def vec_uppertri(M):
    iu = np.triu_indices_from(M, k=1)
    return M[iu]

def fc_vector_from_npy(path):
    X = np.load(path)
    C = np.corrcoef(X, rowvar=False)
    Cz = fisher_z(C)
    return vec_uppertri(Cz)

def build_fc_matrix(df):
    feats = [fc_vector_from_npy(p) for p in df['npy_path']]
    X = np.vstack(feats)
    y = df['label'].values.astype(int)
    return X, y

def pid2site(pid):
    s = pid.replace('sub-','')
    m = re.match(r'([a-z]+)', s)
    return m.group(1) if m else 'na'

In [17]:
if RUN_BASELINES and DO_LOSO:
    sites = sorted({pid2site(pid) for pid in data_df['participant_id']})
    res = []
    for site in sites:
        tr_idx = data_df['participant_id'].map(pid2site) != site
        va_idx = ~tr_idx
        tr_df = data_df[tr_idx].reset_index(drop=True)
        va_df = data_df[va_idx].reset_index(drop=True)
        if len(va_df) < 5 or len(tr_df) < 10:
            print(f"[LOSO:{site}] слишком мало данных — пропуск")
            continue
        # Формирование признаков (могут содержать NaN из-за нулевой дисперсии ROI)
        Xtr, ytr = build_fc_matrix(tr_df)
        Xva, yva = build_fc_matrix(va_df)
        # Заменяем NaN / inf значениями 0 перед любыми трансформациями
        Xtr = np.nan_to_num(Xtr, nan=0.0, posinf=0.0, neginf=0.0)
        Xva = np.nan_to_num(Xva, nan=0.0, posinf=0.0, neginf=0.0)
        # Масштабирование
        scaler = StandardScaler()
        Xtr_s = scaler.fit_transform(Xtr)
        Xva_s = scaler.transform(Xva)
        # Повторная санитаризация (иногда стандартизация может породить inf при нулевом std)
        Xtr_s = np.nan_to_num(Xtr_s, nan=0.0, posinf=0.0, neginf=0.0)
        Xva_s = np.nan_to_num(Xva_s, nan=0.0, posinf=0.0, neginf=0.0)
        # PCA (с проверкой допустимого числа компонент и отсутствия NaN)
        if FC_PCA_COMPONENTS and FC_PCA_COMPONENTS > 0:
            allowable = min(Xtr_s.shape[0], Xtr_s.shape[1])
            n_comp = min(FC_PCA_COMPONENTS, allowable)
            if n_comp < 1:
                print(f"[LOSO:{site}] Пропуск PCA (недостаточно компонент).")
            else:
                if n_comp != FC_PCA_COMPONENTS:
                    print(f"[LOSO:{site}] PCA компонентов снижено до {n_comp} (allowable={allowable}).")
                # Санитизация перед PCA fit
                Xtr_s = np.nan_to_num(Xtr_s, nan=0.0, posinf=0.0, neginf=0.0)
                Xva_s = np.nan_to_num(Xva_s, nan=0.0, posinf=0.0, neginf=0.0)
                try:
                    pca = PCA(n_components=n_comp)
                    Xtr_s = pca.fit_transform(Xtr_s)
                    Xva_s = pca.transform(Xva_s)
                    # Финальная санитаризация
                    Xtr_s = np.nan_to_num(Xtr_s, nan=0.0, posinf=0.0, neginf=0.0)
                    Xva_s = np.nan_to_num(Xva_s, nan=0.0, posinf=0.0, neginf=0.0)
                except ValueError as e:
                    print(f"[LOSO:{site}] Пропуск PCA из-за ошибки: {e}")
        # Обучение логистической регрессии
        clf = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs')
        clf.fit(Xtr_s, ytr)
        pr = clf.predict_proba(Xva_s); yh = pr.argmax(1)
        acc = accuracy_score(yva, yh)
        f1  = f1_score(yva, yh, average='macro')
        try:
            auc = roc_auc_score(yva, pr, multi_class='ovr', average='macro')
        except Exception:
            auc = float('nan')
        res.append({"site": site, "N_val": len(va_df), "acc":acc, "auc":auc, "f1":f1})
        print(f"[LOSO:{site}] N={len(va_df)} | acc={acc:.3f} | auc={auc:.3f} | f1={f1:.3f}")
    if res:
        df_res = pd.DataFrame(res)
        display(df_res)
        print("Средние по сайтам:", df_res[['acc','auc','f1']].mean().to_dict())


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[LOSO:kki] PCA компонентов снижено до 156 (allowable=156).


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[LOSO:kki] N=9 | acc=0.667 | auc=nan | f1=0.267


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


...

/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[LOSO:washu] PCA компонентов снижено до 147 (allowable=147).
[LOSO:washu] N=18 | acc=1.000 | auc=nan | f1=1.000


/usr/local/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


,site,N_val,acc,auc,f1
0,kki,9,0.666667,NaN,0.266667
1,neuroimage,14,0.571429,0.638738,0.344444
2,nyu,65,0.523077,NaN,0.228956
3,peking,23,0.652174,NaN,0.263158
4,pittsburgh,36,0.944444,NaN,0.485714
5,washu,18,1.000000,NaN,1.000000


Средние по сайтам: {'acc': 0.7262984197766807, 'auc': 0.6387377173091459, 'f1': 0.4314899200864113}
